# Segmenting and Clustering Neighborhoods in Austin 

## IBM Applied Data Science Capstone

#### Andriy Zakharko, 2020

<p align='center'>
    <img src='https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fupload.wikimedia.org%2Fwikipedia%2Fcommons%2Fa%2Fa5%2FAustin_Evening.jpg&f=1&nofb=1'>
    </img>
</p>

1. Data for Austin Locations (Geo, ZIP Codes, Names) was carefuly mined from https://zipdatamaps.com/ and stored in Austin_Zip_Loc.csv file
2. Notebook is self-explanatory
3. If you wnant to try it out - you need to register in foursquare developer portal - https://developer.foursquare.com/ and retrieve your own key and secret, and then use it in the Notebook (beware of maps slowdown - it's a lot on the map to render and so lags are possible)

Importing Libraries, defining constants and do all the rest of preparations

In [79]:
import pandas as pd
import requests
import folium

CLIENT_ID       = '__PLACEHOLDER__' # your Foursquare ID
CLIENT_SECRET   = '__PLACEHOLDER__' # your Foursquare Secret
VERSION         = '20200707' # Foursquare API version
LIMIT           = 200 # limit of number of venues returned by Foursquare API
RADIUS          = 1000 # define radius

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("done..")

done..


Read csv with Austin data: Neighborhood Name, Longitude/Latitude and Zip

In [80]:
geo_df = pd.read_csv("Austin_Zip_Loc.csv")
geo_df.head(10)

Name  Longitude   Latitude    Zip
0    Cedar Park -97.808197  30.504881  78613
1     Del Valle -97.587593  30.162531  78617
2       Leander -97.913300  30.512609  78641
3         Manor -97.523033  30.346609  78653
4  Pflugerville -97.597473  30.440781  78660
5    Round Rock -97.720253  30.512060  78681
6        Austin -97.740601  30.267151  78701
7        Austin -97.713600  30.264250  78702
8        Austin -97.767548  30.296049  78703
9        Austin -97.771751  30.241400  78704

Let's put it on the map

In [81]:
_map = folium.Map(location=[ 30.30, -97.70 ], zoom_start=10, tiles='OpenStreetMap')
incidents = folium.map.FeatureGroup()

for lat, lng, name, zip_code in zip(geo_df.Latitude, geo_df.Longitude, geo_df.Name, geo_df.Zip):
    folium.Marker(
        location = [lat, lng],
        popup = name, 
        tooltip = zip_code,
        icon = folium.Icon(color='red')
    ).add_to(_map)

# add incidents to map
_map.add_child(incidents)

Lets define out foursquare API processing function:

In [82]:
def getNearbyVenues(names, zip_codes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, zip_code, lat, lng in zip(names, zip_codes, latitudes, longitudes):
        # print(name) # for debug
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            zip_code,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Zip Code',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

print('done..')

done..


And let's use it to grub venues data:

In [83]:
austin_venues = getNearbyVenues(names=geo_df['Name'], zip_codes=geo_df['Zip'], latitudes=geo_df['Latitude'], longitudes=geo_df['Longitude'])
austin_venues.head(10)

Neighborhood  Zip Code  Neighborhood Latitude  Neighborhood Longitude  \
0   Cedar Park     78613              30.504881              -97.808197   
1   Cedar Park     78613              30.504881              -97.808197   
2   Cedar Park     78613              30.504881              -97.808197   
3   Cedar Park     78613              30.504881              -97.808197   
4   Cedar Park     78613              30.504881              -97.808197   
5   Cedar Park     78613              30.504881              -97.808197   
6   Cedar Park     78613              30.504881              -97.808197   
7   Cedar Park     78613              30.504881              -97.808197   
8   Cedar Park     78613              30.504881              -97.808197   
9   Cedar Park     78613              30.504881              -97.808197   

                                   Venue  Venue Latitude  Venue Longitude  \
0            Brushy Creek Regional Trail       30.496626       -97.809235   
1                          Jardin Corona       30.504275       -97.818268   
2  U-Haul Moving & Storage of Cedar Park       30.503569       -97.816988   
3                               Circle K       30.500993       -97.815452   
4                           CVS pharmacy       30.500957       -97.817331   
5                   Brushy Creek and 183       30.504870       -97.809675   
6                                   AT&T       30.502567       -97.806792   
7          Peak Performance Martial Arts       30.504431       -97.813236   
8     Square Cow Movers & Storage Austin       30.500343       -97.804394   
9     the bungalows hotel & event center       30.499228       -97.813708   

       Venue Category  
0               Trail  
1  Mexican Restaurant  
2    Storage Facility  
3   Convenience Store  
4            Pharmacy  
5       Moving Target  
6   Electronics Store  
7   Martial Arts Dojo  
8        Home Service  
9               Hotel

Ok, let's add them to our map and enrich our zip-code-based markers

In [84]:
# _map = folium.Map(location=[ 30.30, -97.70 ], zoom_start=10, tiles='OpenStreetMap')
incidents = folium.map.FeatureGroup()

for lat, lng, name, category in zip(austin_venues['Venue Latitude'], austin_venues['Venue Longitude'], austin_venues['Venue'], austin_venues['Venue Category']):
    folium.Marker(
        location = [lat, lng],
        popup = name, 
        tooltip = category,
        icon = folium.Icon(color='blue')
    ).add_to(_map)

# add incidents to map
_map.add_child(incidents)

Do some exploratory aggregations:

In [85]:
aggregate = austin_venues[['Neighborhood', 'Venue Category']]
aggregate.groupby("Neighborhood").count()

Venue Category
Neighborhood                
Austin                  1111
Cedar Park                13
Leander                    2
Manor                      5
Pflugerville               4
Round Rock                10

Ok - it's clear that 99% of all venues lay down in one place, which is not very useful for data understanding. 
And no the map
Let's re-shape it and add Zip Code to our model so we can bind it more specifically:

In [86]:
aggregate = austin_venues[['Zip Code', 'Venue Category']]
result = aggregate.groupby("Zip Code").count().sort_values(by='Venue Category', ascending=False).head(10)
result

Venue Category
Zip Code                
78705                100
78757                100
78701                100
78756                 90
78751                 82
78722                 73
78702                 72
78712                 53
78704                 50
78748                 48

Ok, looks like now we have clear locations where venues are concentrated in Austin. Let's continue exploring your data. How many different unique venues do we have in Austin?

In [87]:
len(austin_venues['Venue Category'].unique())

238

In [88]:
austin_venues.groupby('Zip Code').count().head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  Venue  \
Zip Code                                                                       
78613               13                     13                      13     13   
78641                2                      2                       2      2   
78653                5                      5                       5      5   
78660                4                      4                       4      4   
78681               10                     10                      10     10   

          Venue Latitude  Venue Longitude  Venue Category  
Zip Code                                                   
78613                 13               13              13  
78641                  2                2               2  
78653                  5                5               5  
78660                  4                4               4  
78681                 10               10              10

In [89]:
to_onehot = pd.get_dummies(austin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Zip Codes'] = austin_venues['Zip Code'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1145, 239)


Zip Codes  Adult Boutique  Airport Service  American Restaurant  \
0      78613               0                0                    0   
1      78613               0                0                    0   
2      78613               0                0                    0   
3      78613               0                0                    0   
4      78613               0                0                    0   

   Antique Shop  Arcade  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0       0            0           0                    0   
1             0       0            0           0                    0   
2             0       0            0           0                    0   
3             0       0            0           0                    0   
4             0       0            0           0                    0   

   Arts & Entertainment  Asian Restaurant  Assisted Living  \
0                     0                 0                0   
1                     0                 0                0   
2                     0                 0                0   
3                     0                 0                0   
4                     0                 0                0   

   Athletics & Sports  Auto Dealership  Auto Garage  Automotive Shop  \
0                   0                0            0                0   
1                   0                0            0                0   
2                   0                0            0                0   
3                   0                0            0                0   
4                   0                0            0                0   

   BBQ Joint  Bagel Shop  Bakery  Bank  Bar  Baseball Field  Basketball Court  \
0          0           0       0     0    0               0                 0   
1          0           0       0     0    0               0                 0   
2          0           0       0     0    0               0                 0   
3          0           0       0     0    0               0                 0   
4          0           0       0     0    0               0                 0   

   Beer Bar  Beer Garden  Beer Store  Bookstore  Boutique  Breakfast Spot  \
0         0            0           0          0         0               0   
1         0            0           0          0         0               0   
2         0            0           0          0         0               0   
3         0            0           0          0         0               0   
4         0            0           0          0         0               0   

   Brewery  Bubble Tea Shop  Burger Joint  Burrito Place  Bus Station  \
0        0                0             0              0            0   
1        0                0             0              0            0   
2        0                0             0              0            0   
3        0                0             0              0            0   
4        0                0             0              0            0   

   Bus Stop  Business Service  Butcher  Cafeteria  Café  \
0         0                 0        0          0     0   
1         0                 0        0          0     0   
2         0                 0        0          0     0   
3         0                 0        0          0     0   
4         0                 0        0          0     0   

   Cajun / Creole Restaurant  Camera Store  Capitol Building  Cheese Shop  \
0                          0             0                 0            0   
1                          0             0                 0            0   
2                          0             0                 0            0   
3                          0             0                 0            0   
4                          0             0                 0            0   

   Chinese Restaurant  Churrascaria  Clothing Store  Cocktail Bar  \
0                   0             0               0          

In [90]:
to_grouped = to_onehot.groupby(["Zip Codes"]).mean().reset_index()

print(to_grouped.shape)
to_grouped.head()

(45, 239)


Zip Codes  Adult Boutique  Airport Service  American Restaurant  \
0      78613             0.0              0.0                  0.0   
1      78641             0.0              0.0                  0.0   
2      78653             0.0              0.0                  0.0   
3      78660             0.0              0.0                  0.0   
4      78681             0.0              0.0                  0.0   

   Antique Shop  Arcade  Art Gallery  Art Museum  Arts & Crafts Store  \
0           0.0     0.0          0.0         0.0                  0.0   
1           0.0     0.0          0.0         0.0                  0.0   
2           0.0     0.0          0.0         0.0                  0.0   
3           0.0     0.0          0.0         0.0                  0.0   
4           0.0     0.0          0.0         0.0                  0.0   

   Arts & Entertainment  Asian Restaurant  Assisted Living  \
0                   0.0               0.0              0.0   
1                   0.0               0.0              0.0   
2                   0.0               0.0              0.0   
3                   0.0               0.0              0.0   
4                   0.0               0.0              0.0   

   Athletics & Sports  Auto Dealership  Auto Garage  Automotive Shop  \
0                 0.0              0.0          0.0              0.0   
1                 0.0              0.0          0.0              0.0   
2                 0.0              0.0          0.0              0.0   
3                 0.0              0.0          0.0              0.0   
4                 0.0              0.0          0.0              0.0   

   BBQ Joint  Bagel Shop  Bakery  Bank  Bar  Baseball Field  Basketball Court  \
0        0.0         0.0     0.0   0.0  0.0             0.0               0.0   
1        0.0         0.0     0.0   0.0  0.0             0.0               0.0   
2        0.0         0.0     0.0   0.0  0.2             0.0               0.0   
3        0.0         0.0     0.0   0.0  0.0             0.0               0.0   
4        0.0         0.0     0.0   0.1  0.0             0.0               0.0   

   Beer Bar  Beer Garden  Beer Store  Bookstore  Boutique  Breakfast Spot  \
0       0.0          0.0         0.0        0.0       0.0             0.0   
1       0.0          0.0         0.0        0.0       0.0             0.0   
2       0.0          0.0         0.0        0.0       0.0             0.0   
3       0.0          0.0         0.0        0.0       0.0             0.0   
4       0.0          0.0         0.0        0.0       0.0             0.0   

   Brewery  Bubble Tea Shop  Burger Joint  Burrito Place  Bus Station  \
0      0.0              0.0           0.0            0.0          0.0   
1      0.0              0.0           0.0            0.0          0.0   
2      0.0              0.0           0.0            0.0          0.0   
3      0.0              0.0           0.0            0.0          0.0   
4      0.0              0.0           0.0            0.0          0.0   

   Bus Stop  Business Service  Butcher  Cafeteria  Café  \
0       0.0               0.0      0.0        0.0   0.0   
1       0.0               0.0      0.0        0.0   0.0   
2       0.0               0.0      0.0        0.0   0.0   
3       0.0               0.0      0.0        0.0   0.0   
4       0.0               0.0      0.0        0.0   0.0   

   Cajun / Creole Restaurant  Camera Store  Capitol Building  Cheese Shop  \
0                        0.0           0.0               0.0          0.0   
1                        0.0           0.0               0.0          0.0   
2                        0.0           0.0               0.0          0.0   
3                        0.0           0.0               0.0          0.0   
4                        0.0           0.0               0.0          0.0   

   Chinese Restaurant  Churrascaria  Clothing Store  Cocktail Bar  \
0                 0.0           0.0             0.0          

In [91]:
#to_trails = to_grouped[["Zip Codes","Trail"]]
#to_trails.head()
to_parks = to_grouped[['Zip Codes', 'Park']]
to_parks.head()

Zip Codes  Park
0      78613   0.0
1      78641   0.0
2      78653   0.0
3      78660   0.0
4      78681   0.1

Now let's applay clustering:

In [92]:
from sklearn.cluster import KMeans
K = 7

to_clustering = to_parks.drop(["Zip Codes"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=K, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:20]

to_merged = to_parks.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged.head()

Zip Codes  Park  Cluster Labels
0      78613   0.0               1
1      78641   0.0               1
2      78653   0.0               1
3      78660   0.0               1
4      78681   0.1               4

In [93]:
to_merged = to_merged.join(austin_venues.set_index("Zip Code"), on="Zip Codes")

print(to_merged.shape)
to_merged.head()

(1145, 10)


Zip Codes  Park  Cluster Labels Neighborhood  Neighborhood Latitude  \
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   

   Neighborhood Longitude                                  Venue  \
0              -97.808197            Brushy Creek Regional Trail   
0              -97.808197                          Jardin Corona   
0              -97.808197  U-Haul Moving & Storage of Cedar Park   
0              -97.808197                               Circle K   
0              -97.808197                           CVS pharmacy   

   Venue Latitude  Venue Longitude      Venue Category  
0       30.496626       -97.809235               Trail  
0       30.504275       -97.818268  Mexican Restaurant  
0       30.503569       -97.816988    Storage Facility  
0       30.500993       -97.815452   Convenience Store  
0       30.500957       -97.817331            Pharmacy

Ok, when we have our claster labels in dataset, let's color it on the map

In [134]:
map_clusters = folium.Map(location=[ 30.30, -97.70 ], zoom_start=10, tiles='OpenStreetMap')

markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'cyan'
markers_colors[3] = 'green'
markers_colors[4] = 'yellow'
markers_colors[5] = 'lightred'
markers_colors[6] = 'darkgreen'


for lat, lng, cluster_label in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    folium.Marker(
        location = [lat, lng],
        tooltip = cluster_label,
        icon = folium.Icon(color=markers_colors[cluster_label])
    ).add_to(map_clusters)

# add incidents to map
map_clusters.add_child(incidents)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: color argument of Icon should be one of: {'blue', 'purple', 'cadetblue', 'lightred', 'gray', 'beige', 'orange', 'darkpurple', 'white', 'lightgray', 'black', 'green', 'darkblue', 'darkgreen', 'pink', 'lightgreen', 'red', 'lightblue', 'darkred'}.


Some more clustering details:

In [96]:
kmeans.cluster_centers_

array([[0.21111111],
       [0.00097222],
       [0.42857143],
       [0.04166706],
       [0.08846154],
       [0.14285714],
       [0.02589375]])

In [98]:
to_merged.shape

(1145, 10)

In [102]:
to_merged.head()

Zip Codes  Park  Cluster Labels Neighborhood  Neighborhood Latitude  \
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   
0      78613   0.0               1   Cedar Park              30.504881   

   Neighborhood Longitude                                  Venue  \
0              -97.808197            Brushy Creek Regional Trail   
0              -97.808197                          Jardin Corona   
0              -97.808197  U-Haul Moving & Storage of Cedar Park   
0              -97.808197                               Circle K   
0              -97.808197                           CVS pharmacy   

   Venue Latitude  Venue Longitude      Venue Category  
0       30.496626       -97.809235               Trail  
0       30.504275       -97.818268  Mexican Restaurant  
0       30.503569       -97.816988    Storage Facility  
0       30.500993       -97.815452   Convenience Store  
0       30.500957       -97.817331            Pharmacy

Finally, let's describe our clasters and highlight what's in each cluster (top 5 venues)

In [133]:
groups = to_merged[['Cluster Labels', 'Venue Category']].groupby("Cluster Labels")
for group in groups:
    print("label#: " + str(group[0]))
    print(group[1]['Venue Category'].unique()[0:5])
    print('')

label#: 0
['Park' 'Recreation Center' 'Coffee Shop' 'Night Market'
 'Gym / Fitness Center']

label#: 1
['Trail' 'Mexican Restaurant' 'Storage Facility' 'Convenience Store'
 'Pharmacy']

label#: 2
['Park' 'Business Service' "Doctor's Office" 'Pool']

label#: 3
['Taco Place' 'Coffee Shop' 'Pub' 'Cocktail Bar'
 'South American Restaurant']

label#: 4
['Pharmacy' 'Bank' 'Video Store' 'Park' 'Intersection']

label#: 5
['Park' 'Pool' 'Playground' 'Tennis Court'
 'Residential Building (Apartment / Condo)']

label#: 6
['Coffee Shop' 'Fast Food Restaurant' 'Sandwich Place'
 'Mexican Restaurant' 'American Restaurant']



## Summary

<hr>
This is the end of the technical part of the project. Please find the rest in the project final report and thepresentation.